In [ ]:
from discovery_utils.getters import gtr
from discovery_utils.getters import crunchbase

from src import VECTOR_DB_DIR
from src import search
from src import PROJECT_DIR

OUTPUT_DIR = PROJECT_DIR / 'data/searches/01_cybersec'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


## Gateway to Research search

In [ ]:
GTR = gtr.GtrGetter(vector_db_path=VECTOR_DB_DIR)

In [17]:
Search = search.SearchDataset(GTR, GTR.projects_enriched, "config.yaml")

In [ ]:
search_df = Search.do_search()

In [ ]:
search_df.head(50)[['title', 'url', '_score_keywords', '_score_vectors', '_score']]


In [20]:
search_df.to_csv(OUTPUT_DIR / 'gtr_search_results.csv', index=False)

## Crunchbase search

In [ ]:
CB = crunchbase.CrunchbaseGetter(vector_db_path=VECTOR_DB_DIR)

In [21]:
SearchCB = search.SearchDataset(CB, CB.organisations_enriched, "config.yaml")

In [ ]:
search_cb_df = SearchCB.do_search()

In [ ]:
search_cb_df.head(50)[['name', 'short_description', 'homepage_url', '_score_keywords', '_score_vectors', '_score']]


In [24]:
search_cb_df.to_csv(OUTPUT_DIR / 'cruchbase_search_results.csv', index=False)